In [1]:
%matplotlib inline 
%load_ext autoreload
%autoreload 2

import time
from approxeng.input.selectbinder import ControllerResource
from IPython.display import display, clear_output
from jetcam.utils import bgr8_to_jpeg
import numpy as np
import cv2
import ipywidgets
import serial

In [3]:
from camera import Camera
from CV_FIND import CV_IMG_PROCESSOR

width = int(244)
height = int(244)
    
camera = Camera(width=width, height=height, capture_width=1280, capture_height=720, capture_fps=60, capture_flip=2)
main_cv = CV_IMG_PROCESSOR()

In [10]:
BENCHMARK = True
SHOW_LIVE = True
RECORD = True
Frames = []

if(SHOW_LIVE):
    widget_run = ipywidgets.Image(format='jpeg', width=width, height=height/2)
    display(widget_run)

if(BENCHMARK):
    times = []
    total_start_time = time.time()
        
def update_images():
    img = camera.value
    main_cv.getSteering(img,"canny")    
    main_cv.raw_img = img
        
with serial.Serial('/dev/ttyUSB0', 1000000, timeout=1) as ser:
    with ControllerResource() as joystick:
        
        CONST_TIME = 0.017
        P = 4
        D = 0.02
        error_old = 0
        STEERING = 90
        THROTTLE = int(90)
        START = 0
        start_time = 0
        cumulative_times = []
        speeds = []
        
        
        while joystick.connected:
            if(joystick['cross'] is not None):  # SQUARE EXIT
                output = "{:05d}-{:05d}\n".format(int(90), int(90))
                ser.write(bytes(output,'utf-8'))
                print("EXIT")
                ser.close()
                break
                
            if(BENCHMARK):
                fps_timer = time.time()
                
            # PD
            error = (main_cv.angle-90) # -90 0 90
            P_T = error * P
            D_T = ((error-error_old)/CONST_TIME)*D
            STEERING = int(np.clip(((P_T-D_T)+90), 2, 178))
            error_old = error
            
            if(joystick['square'] is not None):# LB for timer control
                if(START):
                    cumulative_times.append((time.time()-start_time))
                    print("Pause timer", len(lap_times))
                    START = 0 
                else:
                    print("Start timer")
                    start_time = time.time()
                    START = 1
                time.sleep(0.2)
                
            #if(START):
                # negative for auto throttle
            #    THROTTLE = int(-95)
            #else:
            #STEERING = int((joystick['rx']+1)/2*180)
            THROTTLE = int((joystick['ly']+1)/2*180)
            
            # Some manual 
            if(joystick['l1'] is not None):# L trigger for manual
                STEERING = int((joystick['rx']+1)/2*180)
                THROTTLE = int((joystick['ly']+1)/2*180)
            
            update_images()
                
            if(SHOW_LIVE):  
                output = "{:05d}-{:05d}\n".format(int(THROTTLE), int(STEERING))
                ser.write(bytes(output,'utf-8'))
                speed = (ser.readline())#.decode('utf-8').strip()   # read a '\n' terminated line
                if not speed:
                    speed = 0
                    
                info_str = "STEERING {:1d}, THROTTLE {:1d}, SPEED {:1d}".format(int(STEERING), int(abs(THROTTLE)), int(speed))
                preview = main_cv.preview_img
                cv2.putText(preview, info_str, (0, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255))
                widget_run.value = bgr8_to_jpeg(preview)
                if(RECORD and START):
                    Frames.append(preview)
                    speeds.append(speed)
                
            if(BENCHMARK):
                delta = (time.time() - fps_timer)
                times.append(delta)

if(BENCHMARK):
    mean_delta = np.array(times).mean()
    fps = 1 / mean_delta
    total_time = time.time()-total_start_time
    print('average(sec):{:.2f}, fps:{:.2f}, time:{:.2f}'.format(mean_delta, fps, total_time))

Image(value=b'', format='jpeg', height='122.0', width='244')

EXIT
average(sec):0.01, fps:117.59, time:116.12


In [82]:
# OOPS
print(cumulative_times)
print("TOTAL", sum(cumulative_times))
print(speeds[100])
result = []
for b in speeds:
    result.append(int(b))
    
print(np.mean(result))

[69.48931622505188]
TOTAL 69.48931622505188
b'66\r\n'
87.67291311754686


In [81]:
preview_widget = ipywidgets.Image(format='jpeg', width=width, height=height/2)
display(preview_widget)
#out = cv2.VideoWriter('pid_onboard.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 100, (width,height))
out = cv2.VideoWriter("pid.avi",cv2.VideoWriter_fourcc(*"MJPG"), 120,(244,244),0)
for f in Frames:
    out.write(f.astype('uint8'))
    preview_widget.value = bgr8_to_jpeg(f)
    
out.release()

Image(value=b'', format='jpeg', height='122.0', width='244')

In [11]:
camera.unobserve_all() 
camera.stop()
Frames.clear()